In [14]:
import numpy as np
import networkx as nx
from networkx.generators.community import LFR_benchmark_graph, planted_partition_graph,random_partition_graph
from networkx.algorithms import community
import os
import math

import functools

In [9]:
a = '5 8 20 1 7 37 2 35 13 39 9 45 47 38 26 10 33 48 278 16 18 492 3 467 223 101 19 80 44 295 210 285 230 108 42 376 117 221 476 172 425 328 448 371 432 495 412 288 14 6 12 24 30 23 28 29 4 34 22 41 17 21 0 40 27 15 25 36 31 11 32 43 46 49 415 296 493 273 212 181 263 407 445 140 271 185 488 229 475 431 387 262 226 200 473 162 442 259 353 440 420 292 456 234 480 404 419 267 449 110 479 287 217 182 459 208 275 102 487 149 146 457 468 231 126 236 125 336 279 298 169 111 107 497 254 302 167 148 160 284 257 258 256 242 274 86 499 471 414 131 438 269 206 264 213 439 243 335 112 470 127 139 341 489 400 225 294 461 481 250 358 385 268 441 201 109 240 485 458 482 216 355 486 477 444 299 189 142 409 351 394 270 484 164 454 188 399 115 84 98 144 397 411 369 281 421 239 289 260 405 406 436 152 233 388 277 261 203 318 235 85 66 248 241 291 396 498 272 265 408 465 446 321 426 362 462 428 455 391 136 401 280 314 354 50 207 177 443 249 380 '
b = a.strip().split(' ')[:-1]

list(map(lambda x: int(x), b))


[5,
 8,
 20,
 1,
 7,
 37,
 2,
 35,
 13,
 39,
 9,
 45,
 47,
 38,
 26,
 10,
 33,
 48,
 278,
 16,
 18,
 492,
 3,
 467,
 223,
 101,
 19,
 80,
 44,
 295,
 210,
 285,
 230,
 108,
 42,
 376,
 117,
 221,
 476,
 172,
 425,
 328,
 448,
 371,
 432,
 495,
 412,
 288,
 14,
 6,
 12,
 24,
 30,
 23,
 28,
 29,
 4,
 34,
 22,
 41,
 17,
 21,
 0,
 40,
 27,
 15,
 25,
 36,
 31,
 11,
 32,
 43,
 46,
 49,
 415,
 296,
 493,
 273,
 212,
 181,
 263,
 407,
 445,
 140,
 271,
 185,
 488,
 229,
 475,
 431,
 387,
 262,
 226,
 200,
 473,
 162,
 442,
 259,
 353,
 440,
 420,
 292,
 456,
 234,
 480,
 404,
 419,
 267,
 449,
 110,
 479,
 287,
 217,
 182,
 459,
 208,
 275,
 102,
 487,
 149,
 146,
 457,
 468,
 231,
 126,
 236,
 125,
 336,
 279,
 298,
 169,
 111,
 107,
 497,
 254,
 302,
 167,
 148,
 160,
 284,
 257,
 258,
 256,
 242,
 274,
 86,
 499,
 471,
 414,
 131,
 438,
 269,
 206,
 264,
 213,
 439,
 243,
 335,
 112,
 470,
 127,
 139,
 341,
 489,
 400,
 225,
 294,
 461,
 481,
 250,
 358,
 385,
 268,
 441,
 201,
 109,
 240,


In [3]:
b[:1]

['2']

In [2]:
# number of nodes n1
n1 = 50
k = 10
p = 0.5
mu = 0.5
def get_q(mu, p, k):
    return float((mu * p) / ((1 - mu) * (k - 1)))


# generate undirected unweighted graph
#G_P = planted_partition_graph(l=k, k=n1, p_in=p, p_out=get_q(mu, p, k), seed=0)
G_P = random_partition_graph(sizes = [50 for _ in range(10)],p_in=p, p_out=get_q(mu, p, k), seed=0)
print(f"Generated the planted partition graph with size: {len(G_P)}")


Generated the planted partition graph with size: 500


In [ ]:
G_L = LFR_benchmark_graph()

In [4]:
len(G_P.edges())

12400

In [5]:
# test the first partion one
one_commu = G_P.graph['partition'][0]

# ouput the graph
with open('testout.txt','wb') as f:
    nx.write_edgelist(G_P,f,data=False)

# output the 
with open("testComu.txt",'w') as f:
    for comm in G_P.graph['partition']:
        f.writelines(' '.join(list(map(lambda x: str(x),comm))))
        f.write("\n")

# test 
# read from txt
detected_comm = list(open(
    "/home/sfy/Documents/VScodeProject/SNAPro/project/localmotif/localmotifcluster/outputComu.txt",
    "r",
).read().strip().split(' '))


In [18]:

# compare with F1score
def cal_Fscore(detected_comm, ground_truth_comm, beta=1):
    """
    input : detected_comm: list from read file wiht all str
            ground_truth_comm: list with community of int

    Given a set of algorithmic communities C and the ground truth communities S, F score measures the relevance
    between the algorithmic communities and the ground truth communities.
    F_beta = (1+beta^2) / beta^2 * (precision(S)*recall(S)) / (precision(S)+recall(S))
    """
    # in case the last one is
    if detected_comm: detected_comm = list(map(lambda x: int(x),detected_comm))
    ground_truth_comm = list(ground_truth_comm)
    print(ground_truth_comm)
    correctly_classified = list(set(detected_comm).intersection(set(ground_truth_comm)))
    precision = len(correctly_classified) / float(len((detected_comm)))
    recall = len(correctly_classified) / float(len(ground_truth_comm))
    if precision != 0 and recall != 0:
        Fscore = (
            (1 + math.sqrt(beta))
            / float(math.sqrt(beta))
            * precision
            * recall
            / float(precision + recall)
        )
    else:
        Fscore = 0

    return Fscore


In [5]:
# read communities from file
detected_communities = list(open(
    "/home/sfy/Documents/VScodeProject/SNAPro/project/localmotif/localmotifcluster/outputComu.txt",
    "r",
).read().strip().split(';'))



In [19]:
scoress = []
for index, communities in enumerate(detected_communities):
    comms_seed = communities.split(',')
    # print(comms_seed)
    scores = list(map(functools.partial(cal_Fscore,ground_truth_comm=G_P.graph['partition'][index],beta=1),comms_seed))
    scoress.append(scores)


ValueError: invalid literal for int() with base 10: ' '

In [10]:
def get_score(ground_truth_comms):
    #
    scores= map(cal_Fscore,detected_communities[:len(detected_communities)-1],G_P.graph['partition'])

scores = list(map(cal_Fscore(detected_comm, ground_truth_comm, beta=1),detected_communities[:len(detected_communities)-1]))

get_score(ground_truth_comms = G_P.graph['partition'])

NameError: name 'cal_Fscore' is not defined

In [56]:
cal_Fscore(detected_comm= detected_comm,ground_truth_comm=one_commu)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


0.3322259136212624